In [1]:
import numpy as np 
import pandas as pd 
import os
''''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''


"'for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [2]:
import pandas as pd
from tensorflow.keras.applications import VGG16

In [3]:
vgg16_obj = VGG16(include_top = False, input_shape = (224,224,3))    #  include_top = False is used to skip the layer from flattern
for layer in vgg16_obj.layers:          
    layer.trainable = False
vgg16_obj.summary()

58889256/58889256 [==============================] - 15s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                             

In [4]:
vgg16_obj.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [5]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,Flatten
f1 = Flatten()(vgg16_obj.output)
final_layer = Dense(5, activation='softmax')(f1)
final_layer

<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'dense')>

In [7]:
model = Model(inputs=vgg16_obj.input,outputs=final_layer)


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
crop_datagen = ImageDataGenerator(rescale=1/255,shear_range=0.7,zoom_range=0.5)

In [13]:
path='/Users/HP/kaggle/input/agriculture-crop-images/kag2/'
crop_data =crop_datagen.flow_from_directory(
    directory=path,
    target_size=(224,224),
    batch_size=3,
    class_mode="categorical",
    )

Found 804 images belonging to 5 classes.


In [14]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])   

In [15]:
model.fit_generator(crop_data, epochs=20)

C:\Users\HP\AppData\Local\Temp\ipykernel_20384\1198541881.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(crop_data, epochs=20)


Epoch 1/20
268/268 [==============================] - 250s 929ms/step - loss: 1.9951 - accuracy: 0.4241
Epoch 2/20
268/268 [==============================] - 232s 866ms/step - loss: 1.1438 - accuracy: 0.6381
Epoch 3/20
268/268 [==============================] - 229s 852ms/step - loss: 0.8068 - accuracy: 0.7537
Epoch 4/20
268/268 [==============================] - 234s 871ms/step - loss: 0.6431 - accuracy: 0.7923
Epoch 5/20
268/268 [==============================] - 268s 999ms/step - loss: 0.5792 - accuracy: 0.8209
Epoch 6/20
268/268 [==============================] - 265s 989ms/step - loss: 0.3771 - accuracy: 0.8905
Epoch 7/20
268/268 [==============================] - 251s 937ms/step - loss: 0.3429 - accuracy: 0.9030
Epoch 8/20
268/268 [==============================] - 241s 899ms/step - loss: 0.3320 - accuracy: 0.9005
Epoch 9/20
268/268 [==============================] - 265s 987ms/step - loss: 0.2796 - accuracy: 0.9167
Epoch 10/20
268/268 [==============================] - 264s 983m

In [16]:
def get_pred_res(path):
    A = cv2.imread('/Users/HP/kaggle/input/agriculture-crop-images/test_crop_image/'+file)
    A = cv2.resize(A,(224,224))
    A = A.reshape(1,224,224,3)/255
    y = model.predict(A).argmax()
    if y==0:
        return 'jute'
    elif y==1:
        return 'maize'
    elif y==2:
        return 'rice'
    elif y==3:
        return 'sugarcane'
    else:
        return 'wheat'

In [17]:
Test_result=os.listdir('/Users/HP/kaggle/input/agriculture-crop-images/test_crop_image')

In [18]:
import cv2

In [19]:
Output_res=[]
for file in Test_result:
    Output_res.append([file,get_pred_res(file)])
Output_res

1/1 [==============================] - 0s 302ms/step


[['jute-field.jpg', 'rice'],
 ['jute002.jpg', 'jute'],
 ['jute003.jpg', 'jute'],
 ['jute03.jpg', 'rice'],
 ['jutefield.jpg', 'maize'],
 ['juteimg.jpg', 'rice'],
 ['juteleaves.jpg', 'jute'],
 ['juteplant.jpg', 'maize'],
 ['juteplants.jpg', 'jute'],
 ['maize corn set2.jpg', 'maize'],
 ['maize images.jfif', 'maize'],
 ['maize plant set.jpg', 'wheat'],
 ['maize-Field-Corn.jpg', 'rice'],
 ['maize-field.jpg', 'jute'],
 ['maize-field01.jpg', 'maize'],
 ['maize000.jfif', 'rice'],
 ['maize00corn-fields.jpg', 'maize'],
 ['maize02.jfif', 'maize'],
 ['maizecornleaves.jfif', 'maize'],
 ['maize_fieldmexico.jpeg', 'maize'],
 ['rice-4967164_1280.jpg', 'rice'],
 ['rice-5174887_1280.jpg', 'maize'],
 ['rice-828540_1280.jpg', 'rice'],
 ['rice-field.jpg', 'maize'],
 ['rice-field01.jpg', 'rice'],
 ['rice-field02.jpg', 'rice'],
 ['rice-fields-204128_1280.jpg', 'rice'],
 ['rice8122f869e3f.jpg', 'maize'],
 ['sugarcane fields.jpg', 'maize'],
 ['sugarcane-farm-in-the-mountain-countryside-of-thailand.jpg', 'maize